In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import zipfile
from keras.layers import Dense, Reshape
import matplotlib.pyplot as plt
import math
from keras.losses import *
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape
from keras.layers import LSTM, Flatten, Input, Embedding, concatenate, Concatenate, average, maximum, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
def balanceFraudDataframe(df, p): #p is desired fraud/non-fraud ratio
    dfFraud = df.loc[df['isFraud']==1,:]
    x = len(dfFraud)
    n = len(df)
    numOfTimesToResample = int((p * n - x)/((1-p) * x))
    #print(numOfTimesToResample)
    dfBalanced = df
    for _ in range(numOfTimesToResample):
        dfBalanced = dfBalanced.append(dfFraud)   
    return dfBalanced

In [3]:
def findPercentFraud(df):
    return len(df.loc[df.isFraud == 0, :])/len(df)

In [4]:
df = pd.read_pickle("data/dataframe_train_v06.pd")
df_fraud = pd.read_pickle('dfFraud.pd')

In [5]:
df['isFraud'] = df_fraud['isFraud']

In [6]:
df.isna().sum().sum()

0

In [8]:
#scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler()
scaler.fit(df.drop('isFraud', axis=1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
np.random.seed(1234)
choose = np.random.rand(len(df))
dfTrain = df.loc[choose >= .2, :]
dfTest = df.loc[choose < .2, :]

In [10]:
dfTrain.shape

(472515, 337)

In [11]:
assert len(dfTrain) + len(dfTest) == len(df)

In [12]:
print(findPercentFraud(dfTrain))
dfTrain = balanceFraudDataframe(dfTrain, .15)
print(findPercentFraud(dfTrain))

0.964932330190576
0.8730815261620949


In [13]:
print(findPercentFraud(dfTest))

0.9653209065875874


In [14]:
x_train = scaler.transform(dfTrain.drop(["isFraud"], axis=1))
y_train= np.array(dfTrain['isFraud'])

x_test = scaler.transform(dfTest.drop(["isFraud"], axis=1))
y_test = np.array(dfTest['isFraud'])

In [15]:
from sklearn.utils import class_weight

inputs = Input(shape=(336,))
# We stack a deep densely-connected network on top
x = Dense(600, activation='relu', use_bias=True)(inputs)
x = Dense(600,  activation='relu', use_bias=True)(x)
x = Dropout(.2)(x)
x = Dense(600, activation='relu', use_bias=True)(x)
x = Dropout(.1)(x)
x = Dense(600, activation='relu', use_bias=True)(x)
x = Dropout(.2)(x)
x = Dense(600, activation='relu', use_bias=True)(x)
x = Dense(200, activation='relu', use_bias=True)(x)
# And finally we add the main logistic regression layer
output = Dense(1 , activation='sigmoid', name='ANN_output')(x)

adam = optimizers.Adam(lr=0.001)
model = Model(inputs=inputs, outputs = output)


#weights = class_weight.compute_class_weight('balanced',
#                                            np.unique(y_train),
#                                            y_train)
# weights = [4, 1]
from sklearn.metrics import roc_auc_score

model.compile(loss="binary_crossentropy", optimizer=adam, metrics=['accuracy', ])
model.fit(x_train, y_train, epochs=50, batch_size=1024, verbose=2, )#class_weight= weights)


Epoch 1/50
 - 14s - loss: 0.2326 - acc: 0.9148
Epoch 2/50
 - 13s - loss: 0.1781 - acc: 0.9360
Epoch 3/50
 - 13s - loss: 0.1498 - acc: 0.9458
Epoch 4/50
 - 13s - loss: 0.1295 - acc: 0.9526
Epoch 5/50
 - 13s - loss: 0.1145 - acc: 0.9579
Epoch 6/50
 - 13s - loss: 0.1025 - acc: 0.9616
Epoch 7/50
 - 13s - loss: 0.0930 - acc: 0.9651
Epoch 8/50
 - 13s - loss: 0.0848 - acc: 0.9679
Epoch 9/50
 - 13s - loss: 0.0786 - acc: 0.9699
Epoch 10/50
 - 13s - loss: 0.0725 - acc: 0.9721
Epoch 11/50
 - 13s - loss: 0.0686 - acc: 0.9737
Epoch 12/50
 - 13s - loss: 0.0646 - acc: 0.9749
Epoch 13/50
 - 13s - loss: 0.0614 - acc: 0.9764
Epoch 14/50
 - 13s - loss: 0.0587 - acc: 0.9775
Epoch 15/50
 - 13s - loss: 0.0561 - acc: 0.9782
Epoch 16/50
 - 13s - loss: 0.0530 - acc: 0.9794
Epoch 17/50
 - 13s - loss: 0.0503 - acc: 0.9806
Epoch 18/50
 - 13s - loss: 0.0492 - acc: 0.9809
Epoch 19/50
 - 13s - loss: 0.0472 - acc: 0.9815
Epoch 20/50
 - 13s - loss: 0.0459 - acc: 0.9823
Epoch 21/50
 - 13s - loss: 0.0443 - acc: 0.9828
E

In [24]:
#model.compile(loss="binary_crossentropy", optimizer=adam, metrics=['accuracy' ])

model.fit(x_train, y_train, epochs=50, batch_size=1024, verbose=2, )#class_weight= weights)


Epoch 1/50
 - 13s - loss: 0.0246 - acc: 0.9902
Epoch 2/50
 - 13s - loss: 0.0248 - acc: 0.9903
Epoch 3/50
 - 13s - loss: 0.0237 - acc: 0.9907
Epoch 4/50
 - 13s - loss: 0.0240 - acc: 0.9906
Epoch 5/50
 - 13s - loss: 0.0241 - acc: 0.9906
Epoch 6/50
 - 13s - loss: 0.0239 - acc: 0.9906
Epoch 7/50
 - 13s - loss: 0.0237 - acc: 0.9906
Epoch 8/50
 - 13s - loss: 0.0234 - acc: 0.9908
Epoch 9/50
 - 13s - loss: 0.0234 - acc: 0.9908
Epoch 10/50
 - 13s - loss: 0.0229 - acc: 0.9909
Epoch 11/50
 - 13s - loss: 0.0232 - acc: 0.9910
Epoch 12/50
 - 13s - loss: 0.0228 - acc: 0.9910
Epoch 13/50
 - 13s - loss: 0.0229 - acc: 0.9910
Epoch 14/50
 - 13s - loss: 0.0228 - acc: 0.9911
Epoch 15/50
 - 13s - loss: 0.0229 - acc: 0.9910
Epoch 16/50
 - 13s - loss: 0.0221 - acc: 0.9913
Epoch 17/50
 - 13s - loss: 0.0224 - acc: 0.9912
Epoch 18/50
 - 13s - loss: 0.0221 - acc: 0.9914
Epoch 19/50
 - 13s - loss: 0.0225 - acc: 0.9911
Epoch 20/50
 - 13s - loss: 0.0216 - acc: 0.9916
Epoch 21/50
 - 13s - loss: 0.0220 - acc: 0.9914
E

In [16]:
model.evaluate(x_test, y_test)


118025/118025 [==============================] - 4s 34us/step


[0.13045947130513072, 0.97987714467324383]

In [17]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, model.predict(x_test))

0.93735104391311241

In [21]:
model.save("model_sub2.h5")

In [22]:
df_subm = pd.read_pickle('./data/dataframe_test_v05.pd')
x_sub = scaler.transform(df_subm)
y_pred = model.predict(x_sub)
dfToSubmit = pd.DataFrame(y_pred)
dfToSubmit.columns = ['isFraud']
dfToSubmit.index = df_subm.index
dfToSubmit

,isFraud
TransactionID,
3663549,1.116944e-10
3663550,1.712695e-27
3663551,0.000000e+00
3663552,2.811779e-22
3663553,5.339744e-13
...,...
4170235,2.363108e-08
4170236,4.910627e-27
4170237,6.034514e-33


In [23]:
dfToSubmit.to_csv('./final_submission.csv')

In [ ]:
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train, trainPredict))
#trainScore = numpy.mean(numpy.abs((y_train - trainPredict))/y_train)

print('Train Score: %.2f MAPE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test, testPredict))
#testScore =  numpy.mean(numpy.abs((y_test - testPredict))/y_test)

print('Test Score: %.2f MAPE' % (testScore))




In [6]:
from imblearn.under_sampling import TomekLinks
sampler = TomekLinks()

In [11]:
n = 10
l = df.shape[0]//n
X_ress = []
y_ress = []
for i in range(n):
    print(i)
    X_res, y_res = sampler.fit_resample(df[i*l:(i+1)*l], df_fraud[i*l:(i+1)*l].values.reshape(-1))
    X_ress.append(X_res)
    y_ress.append(y_res)

0
1
2
3
4
5
6
7
8
9


In [12]:
X_res = np.vstack(X_ress)
y_res = np.hstack(y_ress)

In [13]:
X_res.shape

(583698, 716)

In [ ]:
X_res, y_res = sampler.fit_resample(df, df_fraud.values.reshape(-1))

In [9]:
X_res.shape[0]/df.shape[0]

0.9884089138754361

In [10]:
import pickle
with open('./data/dataframe_train_v07.np', 'wb') as f:
    pickle.dump(X_res, f)

In [11]:
with open('./dfFraud.np', 'wb') as f:
    pickle.dump(y_res, f)

In [80]:
df_fraud_res = pd.DataFrame(y_res, X_res[:,0].astype('int32'), df_fraud.columns)#, [str(i) for i in df_fraud.dtypes])

In [79]:
df_res = pd.DataFrame?
#(X_res, X_res[:,0].astype('int32'), df.columns, dtype=

In [ ]:
df_res = pd.DataFrame

In [83]:
for column in df.columns:
    df_res[column] = df_res[column].astype(df[column].dtypes)

In [85]:
df_res.to_pickle('data/dataframe_train_v07.pd')

In [89]:
df_fraud_res.to_pickle('./dfFraud_v07.pd')

TransactionID
2987000       86400
2987001       86401
2987002       86469
2987003       86499
2987004       86506
             ...   
3577535    15811047
3577536    15811049
3577537    15811079
3577538    15811088
3577539    15811131
Name: TransactionDT, Length: 590540, dtype: int64

In [35]:
df.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'